In [ ]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from IPython.display import display
import json
import datetime
import math

from utils.data_exploration_utils import plot_hist, scatterplot, barplots, check_img_resp_cluster_klscore

In [ ]:
# data/processed/2025-08-11_hdbscan/run150

today = datetime.date.today()

proc_dir = config.PROC_DATA_PATH

# folder = "2025-08-12_hdbscan"
# folder_date = folder.split('_')[0]
# run = "run110"
# run = "run69"
folder = "2025-08-11_hdbscan"
folder_date = folder.split('_')[0]
run = "run22"


date = folder.split('_')[0]

img_path = config.SCHULTHESS_DATAPATH
# img_path = os.path.join(img_path, "600x600_imgs")

filepath = os.path.join(proc_dir, "radiographic_features", folder, run)

df = pd.read_csv(os.path.join(filepath, f'{date}_hdbscan_{run}_umap_hdbscan_scaled.csv'))

kl = os.path.join(proc_dir, "2025-08-11_data_exploration", "inmodi_data_questionnaire_kl_woSC.csv")#data/processed/2025-08-11_data_exploration/inmodi_data_questionnaire_kl_woSC.csv
kl = pd.read_csv(kl)

print(f"DF Shape before merge: {df.shape}")
# print(df.columns)
# display(df.head())
# print(kl.columns)
# display(kl[['record_id', 'name']].head())
df = df.merge(kl, left_on='id', right_on='name', how='left')
print(f"DF Shape after merge: {df.shape}")
display(df.head())

# Some Exploration

In [ ]:
df['KL-Score'].value_counts()

In [ ]:
df['cluster_label'].value_counts()

In [ ]:
df = df[df['cluster_label'] != -1]  # remove noise points

In [ ]:
df['cluster_label'].value_counts().reset_index().sort_values(by='cluster_label', ascending=True)

In [ ]:
values = df['cluster_label'].value_counts().reset_index().sort_values(by='cluster_label')

plt.bar(values['cluster_label'], values['count'], color = 'skyblue')
plt.xlabel('Cluster Label')
plt.ylabel('Count')
plt.show()

# Correlation

## Correlation with Questionnaire Scores

In [ ]:
columns_corr = [  
       'oks_q1', 'oks_q2', 'oks_q3', 'oks_q4',
       'oks_q5', 'oks_q6', 'oks_q7', 'oks_q8', 'oks_q9', 'oks_q10', 'oks_q11',
       'oks_q12'
       , 'cluster_label'] 

corr_types = ['spearman']
for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    df_corr = df[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    #plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

In [ ]:
cols = ['koos_s1', 
       'koos_s2', 'koos_s3', 'koos_s4', 'koos_s5', 'koos_s6',
       'koos_s7'
       , 'cluster_label']

corr_types = ['spearman']
for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    df_corr = df[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    #plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

In [ ]:
cols = [
       'koos_p1', 'koos_p2', 'koos_p3', 'koos_p4', 'koos_p5',
       'koos_p6', 'koos_p7', 'koos_p8', 'koos_p9'
       , 'cluster_label']
corr_types = ['spearman']
for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    df_corr = df[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    #plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

In [ ]:
cols = [
       'koos_a1', 'koos_a2',
       'koos_a3', 'koos_a4', 'koos_a5', 'koos_a6', 'koos_a7', 'koos_a8',
       'koos_a9', 'koos_a10', 'koos_a11', 'koos_a12', 'koos_a13', 'koos_a14',
       'koos_a15', 'koos_a16', 'koos_a17'
       , 'cluster_label']
corr_types = ['spearman']
for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    df_corr = df[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    #plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

In [ ]:
cols = [
       'koos_sp1', 'koos_sp2', 'koos_sp3',
       'koos_sp4', 'koos_sp5', 
       'koos_q1', 'koos_q2', 'koos_q3', 'koos_q4'
       , 'cluster_label']
corr_types = ['spearman']
for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    df_corr = df[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    #plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

## Correlation with KL-Score and Pain

In [ ]:
columns_corr = ['pain', 'age',
       'ce_bmi', 'ce_fm',  'cluster_label'] 

corr_types = ['spearman']
for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    df_corr = df[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    #plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

In [ ]:
#'label' is ground truth

df_merged2 = df.copy()
df_merged2 = df_merged2.dropna(subset=['cluster_label', 'KL-Score', 'pain'])

res = stats.spearmanr(df_merged2['cluster_label'].tolist(), df_merged2['KL-Score'].tolist())

print(f"Spearman correlation between cluster label and KL-Score: {res[0]:.3f}")

res = stats.spearmanr(df_merged2['cluster_label'].tolist(), df_merged2['pain'].tolist())
print(f"Spearman correlation between cluster label and pain: {res[0]:.3f}")

In [ ]:
columns_corr.remove('cluster_label')

In [ ]:
columns_corr =  ['pain', 'KL-Score'] 
barplots(df, y_list=columns_corr, x='cluster_label', hue=None, figsize = (6, 6), savepath=None)

In [ ]:
barplots(df, y_list=columns_corr, x='cluster_label', hue='gender', figsize = (6, 6), savepath=None)

# KL-Score Visualization

In [ ]:
labels = list(df['cluster_label'].unique())
labels.sort()

for i in labels:
    df_temp = df.copy()
    df_temp = df_temp[df_temp['cluster_label']==i]
    print(f"For label {i}:")
    display(df_temp['KL-Score'].value_counts().reset_index().sort_values(by="KL-Score"))
    print()

In [ ]:
k=2

## Cluster 0

### KL score 0

In [ ]:
_ = check_img_resp_cluster_klscore(df, cluster_label=0, klscore=0, img_path=img_path, k=k)

### KL-Score 4

In [ ]:
_ = check_img_resp_cluster_klscore(df, cluster_label=0, klscore=4, img_path=img_path, k=k)

## Cluster 3

### KL-Score 0

In [ ]:
try:
    _ = check_img_resp_cluster_klscore(df, cluster_label=3, klscore=0, img_path=img_path, k=k)
except Exception as e:
    print(f"Error: {e}. This cluster might not enough images.")
    try:
        _ = check_img_resp_cluster_klscore(df, cluster_label=3, klscore=0, img_path=img_path)
        
    except Exception as e:
        print(f"Error: {e}. This cluster might not enough images.")


### KL-Score 4

In [ ]:
_ = check_img_resp_cluster_klscore(df, cluster_label=3, klscore=4, img_path=img_path, k=k)

# Get examples

In [ ]:
# # Give me a df showing which clusters have which min max KL-Score
# kl_diffs = df_merged.groupby('cluster_label')['KL-Score'].agg(['min', 'max'])
# kl_diffs.sort_values('max', ascending=False, inplace=True)

In [ ]:
# kl_diffs

In [ ]:
# #for each cluster label give me 2 examples with different KL-Score

# clusters = df_merged['cluster_label'].unique()
# sorted_clusters = sorted(clusters)
# for cluster in sorted_clusters:
#     print(f"Cluster {cluster}:")
#     cluster_df = df_merged[df_merged['cluster_label'] == cluster]
    
#     if len(cluster_df) > 0:
#         for kl_score in cluster_df['KL-Score'].unique():
#             subset = cluster_df[cluster_df['KL-Score'] == kl_score]
#             if len(subset) >= 2:
#                 examples = subset.sample(n=2, random_state=42)
#                 print(f"  KL-Score {kl_score}:")
#                 display(examples[['name', 'id', 'KL-Score', 'pain', 'age', 'ce_bmi', 'ce_fm']])
#             else:
#                 examples = subset
#                 print(f"  KL-Score {kl_score}:")
#                 display(examples[['name', 'id', 'KL-Score', 'pain', 'age', 'ce_bmi', 'ce_fm']])
#     else:
#         print("  No data available for this cluster.")
